In [ ]:
import boto3
import json, uuid, copy, datetime
import random, names

In [ ]:
# open_cmd_in_new_terminal("docker-compose up")
!gnome-terminal -- docker-compose up

In [ ]:
!./setuprds.sh
!./setupdynamo.sh

In [ ]:
!./deploylambdas.sh

In [ ]:
dynamodb = boto3.client('dynamodb', endpoint_url = 'http://localhost:4569')
local_lambda = boto3.client('lambda', endpoint_url = 'http://localhost:4574')

In [ ]:
def print_deployed_functions():
    function_list = local_lambda.list_functions()
    # print(function_list)
    print('Function list: ', [function['FunctionName'] for function in function_list['Functions']])

In [ ]:
def generate_account(client_id = 'zar_client_co'):
    first_name = names.get_first_name()
    family_name = names.get_last_name()
    user_id = str(uuid.uuid4())
    return { 'clientId': client_id, 'ownerUserId': user_id, 'userFirstName': first_name, 'userFamilyName': family_name}

In [ ]:
def decode_lambda_result(lambda_result):
    lambda_payload = lambda_result['Payload'].read()
    lambda_pload_decoded = lambda_payload.decode('utf-8')
    lambda_pload_object = json.loads(lambda_pload_decoded)
    return lambda_pload_object

In [ ]:
def create_number_accounts(number_accounts = 1, client_id = 'zar_client_co'):
    account_dicts = [generate_account(client_id) for i in range(number_accounts)]
    persisted_accounts = []
    for account in account_dicts:
        lambda_result = local_lambda.invoke(FunctionName='create-account', InvocationType='RequestResponse', 
                                           Payload=json.dumps(account))
        lambda_payload = json.loads(decode_lambda_result(lambda_result)['body'])
        persisted_account = copy.deepcopy(account)
        persisted_account['accountId'] = lambda_payload['accountId']
        persisted_account['persistedTime'] = lambda_payload['persistedTime']
        print('Account with ID %s persisted at %s' % (persisted_account['accountId'], persisted_account['persistedTime']))
        persisted_accounts.append(persisted_account)
    
    return persisted_accounts
    

In [ ]:
def generate_saving_transaction(account_id, ref_amount = 100, float_id = 'zar_cash_primary'):
    current_time = datetime.datetime.now()
    saved_amount = round(random.random() * ref_amount * 10 * 10) # random proportion of ref amount, 
    test_saving_dict = { 
        'accountId': account_id, 
        'initiationTime': str(current_time), 
        'settlementTime': str(current_time), 
        'savedAmount': saved_amount, 
        'savedCurrency': 'ZAR', 
        'savedUnit': 'HUNDREDTH_CENT' 
    }
    return test_saving_dict

In [ ]:
def seed_savings_for_accounts(accounts, tx_per_account = 1, base_amount = 100):
    transactions = []
    for i in range(tx_per_account):
        transactions.extend([generate_saving_transaction(account['accountId'], base_amount) for account in accounts])
    # print('transactions: ', transactions)
    tx_records = []
    for tx in transactions:
        lambda_result = local_lambda.invoke(FunctionName='add-savings', InvocationType='RequestResponse', 
                                           Payload=json.dumps(tx, default=str))
        lambda_payload = json.loads(decode_lambda_result(lambda_result)['body'])
        persisted_tx = copy.deepcopy(tx)
        persisted_tx['transactionId'] = lambda_payload['transaction_id']
        persisted_tx['currentBalance'] = lambda_payload['newBalance']
        # persisted_account['persistedTime'] = lambda_payload['persistedTime']
        print('Transaction persisted with ID %s' % (persisted_tx['transactionId']))
        tx_records.append(persisted_tx)
    
    return tx_records

In [ ]:
accounts = create_number_accounts(number_accounts = 2)
print(accounts)

In [ ]:
print_deployed_functions()

In [ ]:
seed_savings_for_accounts(accounts, 1, 100)

In [ ]:
seed_savings_for_accounts(accounts, 1, 100)